In [ ]:
import requests
import json
import datetime

file_name = "your_key_word.json"

with open(file_name, "a") as file:
        file.write("[")

access_token = "your_access_token"
query = "your_key_word"
start_date = 1645660800  
end_date = 1666224000      
start_from = None

params = {
    "access_token": access_token,
    "q": query,
    "start_time": start_date,
    "end_time": end_date,
    "extended": 0,
    "count": 200,
    "v": "5.81",
    "start_from": start_from
}

# make first request to get total number of posts
response = requests.get("https://api.vk.com/method/newsfeed.search", params=params)
data = response.json()

total_count = data["response"]["total_count"]
range_in_hours = 8760 // (total_count // 500)
print("range: ", range_in_hours)
new_start_date = end_date - (range_in_hours * 3600)
print("new start date: ", datetime.date.fromtimestamp(new_start_date))

while True:
    params = {
        "access_token": access_token,
        "q": query,
        "start_time": new_start_date,
        "end_time": end_date,
        "extended": 0,
        "count": 200,
        "v": "5.81",
        "start_from": start_from
    }

    response = requests.get("https://api.vk.com/method/newsfeed.search", params=params)
    data = response.json()
    
    if "response" in data:
        posts = data["response"]["items"]

        for post in posts:
            extracted_post = {
                "id": post["id"],
                "date": post["date"],
                "owner_id": post["owner_id"],
                "from_id": post["from_id"],
                "post_type": post["post_type"],
                "text": post["text"],
                "post_source": {
                    "platform": post.get("post_source", {}).get("platform", ""),
                    "type": post.get("post_source", {}).get("type", ""),
                },
                "comments": {
                    "can_post": post.get("comments", {}).get("can_post", 0),
                    "count": post.get("comments", {}).get("count", 0),
                    "groups_can_post": post.get("comments", {}).get("groups_can_post", 0),
                },
                "likes": {
                    "can_like": post.get("likes", {}).get("can_like", 0),
                    "count": post.get("likes", {}).get("count", 0),
                    "user_likes": post.get("likes", {}).get("user_likes", 0),
                    "can_publish": post.get("likes", {}).get("can_publish", 0),
                },
                "reposts": {
                    "count": post.get("reposts", {}).get("count", 0),
                    "user_reposted": post.get("reposts", {}).get("user_reposted", 0),
                },
                "views": post.get("views", {}).get("count", 0)
            }
            # Saving extracted posts to a JSON file
            with open(file_name, "a") as file:
                json.dump(extracted_post, file, ensure_ascii=False, indent=4)
                file.write(",\n")
            
            new_end_date = post["date"]

    if "next_from" in data["response"]:
        start_from = data["response"]["next_from"]
        print("new start from: ", start_from)
    else:
        start_from = None
        end_date = new_end_date
        # if end_date < 1671753600 and end_date > 1670976000:
        #     print("new end date: ", datetime.date.fromtimestamp(new_end_date))
        #     new_start_date = end_date - (4 * 3600)
        # else:
        print("new end date: ", datetime.date.fromtimestamp(new_end_date))
        new_start_date = end_date - (range_in_hours * 3600)
        print("new start date: ", datetime.date.fromtimestamp(new_start_date))

        if new_start_date < start_date - (range_in_hours * 3600):
            break

print("End")
    
with open(file_name, "a") as file:
        file.write("]")